# Semantic Chunking

### Semantic 분할 기준 방법 (Breakpoint Methods)

의미 기반 청크 분할(Semantic Chunking)은 텍스트 내 문장 간 의미 유사도를 기반으로 분할 지점을 결정하는 방식으로, 단순한 고정 길이 분할에 비해 문맥의 흐름을 더 잘 유지할 수 있다. 이러한 분할 기준을 설정하기 위해 다양한 Breakpoint Method가 활용된다.

---

**1. Percentile 방식:**
문장 간 유사도 차이를 모두 계산한 후, 이 중 X번째 백분위수(percentile)를 기준으로 한다. 해당 기준값보다 유사도 차이가 크게 하락하는 지점을 분할 포인트로 간주한다. 예를 들어, 80번째 백분위수를 설정하면, 상위 20%에 해당하는 급격한 유사도 하락 지점에서 텍스트를 분할하게 된다.

**2. 표준 편차(Standard Deviation):**
전체 유사도의 평균과 표준편차를 계산하여, 평균보다 X 표준편차 이상 낮은 유사도 하락이 발생한 지점을 분할 포인트로 선택한다. 이는 유사도 분포가 정규 분포를 따른다고 가정할 때 통계적으로 이례적인 변화 지점을 포착하는 방식이다.

**3. 사분위 범위(IQR, Interquartile Range):**
유사도 차이의 Q1(제1사분위수)과 Q3(제3사분위수)를 계산하여, IQR = Q3 - Q1 값을 기준으로 한다. Q3 이상 또는 Q1 이하의 극단적인 유사도 하락 구간을 분할 지점으로 설정한다. 이는 이상치를 중심으로 한 분할 기준이다.

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
# PDF 파일에서 텍스트를 추출하는 함수
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일의 경로

    Returns:
        str: 추출된 텍스트 문자열
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 전체 텍스트를 저장할 문자열

    # 각 페이지를 순회하며 텍스트 추출
    for page in mypdf:
        # 현재 페이지에서 텍스트를 추출하고 공백을 추가하여 누적
        all_text += page.get_text("text") + " "

    # 앞뒤 공백을 제거하고 반환
    return all_text.strip()

# PDF 파일 경로 정의
pdf_path = "../../dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트의 처음 500자 출력
print(extracted_text[:500])

인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기


## Creating Sentence-Level Embeddings

In [4]:
# 주어진 텍스트에 대해 임베딩을 생성하는 함수
def get_embedding(text, model="text-embedding-3-small"):
    """
    OpenAI 클라이언트를 사용하여 입력된 텍스트의 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트
        model (str): 사용할 임베딩 모델 이름 (기본값: "BAAI/bge-en-icl")

    Returns:
        np.ndarray: 생성된 임베딩 벡터
    """
    response = client.embeddings.create(model=model, input=text)
    return np.array(response.data[0].embedding)

# 텍스트를 문장 단위로 분할 (기본적인 마침표 기준 분리)
sentences = extracted_text.split(". ")

# 각 문장에 대해 임베딩 벡터 생성
embeddings = [get_embedding(sentence) for sentence in sentences]

print(f"총 {len(embeddings)}개의 문장 임베딩이 생성되었습니다.")

총 257개의 문장 임베딩이 생성되었습니다.


## Calculating Similarity Differences

In [7]:
# 두 벡터 간 코사인 유사도를 계산하는 함수
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
        vec1 (np.ndarray): 첫 번째 벡터
        vec2 (np.ndarray): 두 번째 벡터

    Returns:
        float: 코사인 유사도 값 (범위: -1 ~ 1)
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 인접한 문장 쌍 간 유사도 계산
similarities = [
    cosine_similarity(embeddings[i], embeddings[i + 1]) 
    for i in range(len(embeddings) - 1)
]

## Implementing Semantic Chunking

In [9]:
def compute_breakpoints(similarities, method="percentile", threshold=90):
    """
    유사도 하락에 기반하여 청킹 분할점을 계산합니다.

    Args:
        similarities (List[float]): 문장 간의 유사도 점수 리스트
        method (str): 'percentile', 'standard_deviation', 또는 'interquartile'
        threshold (float): 임계값 (퍼센트일의 경우는 퍼센트, 표준편차의 경우는 표준편차 배수)

    Returns:
        List[int]: 청킹 분할이 발생해야 하는 인덱스 목록
    """
    # 선택된 방법에 따라 임계값을 결정합니다.
    if method == "percentile":
        # 유사도 점수의 X번째 퍼센트일을 계산합니다.
        threshold_value = np.percentile(similarities, threshold)
    elif method == "standard_deviation":
        # 유사도 점수의 평균과 표준편차를 계산합니다.
        mean = np.mean(similarities)
        std_dev = np.std(similarities)
        # 평균에서 X 표준편차를 뺀 값으로 임계값을 설정합니다.
        threshold_value = mean - (threshold * std_dev)
    elif method == "interquartile":
        # 첫 번째와 세 번째 사분위수(Q1 및 Q3)를 계산합니다.
        q1, q3 = np.percentile(similarities, [25, 75])
        # IQR 규칙을 이용해 이상치 기준 임계값을 설정합니다.
        threshold_value = q1 - 1.5 * (q3 - q1)
    else:
        # 유효하지 않은 방법이 제공된 경우 에러를 발생시킵니다.
        raise ValueError("유효하지 않은 방법입니다. 'percentile', 'standard_deviation', 또는 'interquartile' 중에서 선택하세요.")

    # 유사도가 임계값보다 낮은 인덱스를 식별합니다.
    return [i for i, sim in enumerate(similarities) if sim < threshold_value]

# 퍼센트일 방법을 사용하여 임계값 90으로 분할점을 계산합니다.
breakpoints = compute_breakpoints(similarities, method="percentile", threshold=90)

## Splitting Text into Semantic Chunks

In [10]:
def split_into_chunks(sentences, breakpoints):
    """
    문장들을 의미 단위로 분할합니다.

    Args:
        sentences (List[str]): 문장 리스트.
        breakpoints (List[int]): 청킹(분할)이 발생해야 하는 인덱스 목록.

    Returns:
        List[str]: 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트를 초기화합니다.
    start = 0  # 시작 인덱스를 초기화합니다.

    # 각 분할점을 순회하며 청크를 생성합니다.
    for bp in breakpoints:
        # 시작 인덱스부터 현재 분할점까지의 문장들을 연결하여 청크를 생성하고, 청크 리스트에 추가합니다.
        chunks.append(". ".join(sentences[start:bp + 1]) + ".")
        start = bp + 1  # 시작 인덱스를 업데이트하여 다음 문장부터 청크를 생성합니다.

    # 남은 문장들을 마지막 청크로 추가합니다.
    chunks.append(". ".join(sentences[start:]))
    return chunks  # 청크 리스트를 반환합니다.

# split_into_chunks 함수를 사용하여 청크를 생성합니다.
text_chunks = split_into_chunks(sentences, breakpoints)

# 생성된 청크의 개수를 출력합니다.
print(f"의미 청크의 개수: {len(text_chunks)}")

# 결과를 확인하기 위해 첫 번째 청크를 출력합니다.
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

의미 청크의 개수: 231

첫 번째 텍스트 청크:
인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다.


## Creating Embeddings for Semantic Chunks

In [11]:
def create_embeddings(text_chunks):
    """
    각 텍스트 청크에 대한 임베딩을 생성합니다.

    Args:
        text_chunks (List[str]): 텍스트 청크 리스트.

    Returns:
        List[np.ndarray]: 임베딩 벡터 리스트.
    """
    # get_embedding 함수를 사용하여 각 텍스트 청크에 대한 임베딩을 생성합니다.
    return [get_embedding(chunk) for chunk in text_chunks]

# create_embeddings 함수를 사용하여 청크 임베딩을 생성합니다.
chunk_embeddings = create_embeddings(text_chunks)

## Performing Semantic Search

In [12]:
def semantic_search(query, text_chunks, chunk_embeddings, k=5):
    """
    쿼리에 가장 관련성 높은 텍스트 청크들을 찾습니다.

    Args:
        query (str): 검색 쿼리.
        text_chunks (List[str]): 텍스트 청크 리스트.
        chunk_embeddings (List[np.ndarray]): 청크 임베딩 리스트.
        k (int): 반환할 상위 결과의 수.

    Returns:
        List[str]: 상위 k개의 관련성 높은 텍스트 청크 리스트.
    """
    # 쿼리에 대한 임베딩을 생성합니다.
    query_embedding = get_embedding(query)
    
    # 쿼리 임베딩과 각 청크 임베딩 간의 코사인 유사도를 계산합니다.
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 유사도가 높은 순으로 상위 k개의 인덱스를 가져옵니다.
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 상위 k개의 관련성 높은 텍스트 청크들을 반환합니다.
    return [text_chunks[i] for i in top_indices]

In [13]:
# JSON 파일에서 검증 데이터를 불러옵니다.
with open('dataset/validation.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 쿼리를 추출합니다.
query = data[0]['question']

# 관련성 높은 상위 2개의 텍스트 청크를 검색합니다.
top_chunks = semantic_search(query, text_chunks, chunk_embeddings, k=2)

# 쿼리를 출력합니다.
print(f"쿼리: {query}")

# 상위 2개의 관련성 높은 텍스트 청크를 출력합니다.
for i, chunk in enumerate(top_chunks):
    print(f"컨텍스트 {i+1}:\n{chunk}\n{'-'*40}")

쿼리: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
컨텍스트 1:

설명 가능한 AI(XAI) 
설명 가능한 AI(XAI)는 AI 시스템을 보다 투명하고 이해하기 쉽게 만드는 것을 목표로 
합니다. XAI 연구는 AI 결정을 설명하고 신뢰를 높이며 책임감을 향상시키는 방법을 
개발하는 데 중점을 둡니다.
----------------------------------------
컨텍스트 2:

투명성 및 설명 가능성 
투명성과 설명 가능성은 AI 시스템에 대한 신뢰를 구축하는 데 필수적입니다. 설명 가능한 
AI(XAI) 기술은 AI의 결정을 더 이해하기 쉽게 만들어 사용자가 공정성과 정확성을 
평가할 수 있도록 하는 것을 목표로 합니다.
----------------------------------------


## Generating a Response Based on Retrieved Chunks

In [15]:
# AI 어시스턴트를 위한 시스템 프롬프트를 정의합니다.
system_prompt = (
    "당신은 주어진 컨텍스트에 기반하여 엄격하게 대답하는 AI 어시스턴트입니다. "
    "제공된 컨텍스트에서 직접적으로 답변을 도출할 수 없는 경우, 다음과 같이 응답하십시오: "
    "'I do not have enough information to answer that.'"
)

def generate_response(system_prompt, user_message, model="gpt-4o-mini"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델의 응답을 생성합니다.

    Args:
        system_prompt (str): AI의 동작 방식을 안내하는 시스템 프롬프트.
        user_message (str): 사용자 쿼리 또는 메시지.
        model (str): 사용할 AI 모델.

    Returns:
        dict: AI 모델의 응답 객체.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 컨텍스트를 기반으로 사용자 프롬프트를 생성합니다.
user_prompt = "\n".join([
    f"컨텍스트 {i + 1}:\n{chunk}\n-----------------------------\n"
    for i, chunk in enumerate(top_chunks)
])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답을 생성합니다.
ai_response = generate_response(system_prompt, user_prompt)

## Evaluating the AI Response

In [16]:
# 평가 시스템을 위한 시스템 프롬프트를 정의합니다.
evaluate_system_prompt = (
    "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. "
    "AI 어시스턴트의 응답이 정답과 매우 유사하면 점수 1을 부여하세요. "
    "응답이 정답과 맞지 않거나 부적절하다면 점수 0을 부여하세요. "
    "부분적으로 일치하거나 애매한 경우에는 점수 0.5를 부여하세요."
)

# 사용자 쿼리, AI 응답, 정답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
evaluation_prompt = (
    f"사용자 질문: {query}\n"
    f"AI 응답:\n{ai_response.choices[0].message.content}\n"
    f"정답:\n{data[0]['ideal_answer']}\n"
    f"{evaluate_system_prompt}"
)

# 평가 시스템을 통해 평가 응답을 생성합니다.
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 결과를 출력합니다.
print(evaluation_response.choices[0].message.content)

점수 1
